In [246]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from meteostat import Point, Daily

# Set time period
start = datetime(2010, 1, 1)
end = datetime.now() - timedelta(days=1)  # Automatically set to yesterday

# Create Point for Hanoi, Vietnam
hanoi = Point(21.0285, 105.8542, 15)

# Get daily data
df = Daily(hanoi, start, end)
df = df.fetch()




In [247]:
df

,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
time,,,,,,,,,,
2010-01-01,16.3,15.9,17.1,1.3,NaN,45.0,9.7,NaN,NaN,NaN
2010-01-02,17.1,15.4,17.9,0.8,NaN,NaN,7.8,NaN,NaN,NaN
2010-01-03,19.6,16.9,22.6,0.0,NaN,NaN,7.0,NaN,NaN,NaN
2010-01-04,20.1,17.9,22.2,0.0,NaN,108.0,9.2,NaN,NaN,NaN
2010-01-05,22.8,19.8,26.9,0.0,NaN,125.0,9.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2024-11-26,20.9,18.9,23.9,2.3,NaN,2.0,13.3,NaN,1019.0,NaN
2024-11-27,20.4,18.9,21.9,0.0,NaN,3.0,11.5,NaN,1021.3,NaN
2024-11-28,19.7,17.9,21.9,0.0,NaN,354.0,14.2,NaN,1024.2,NaN


In [248]:
df.shape
df.info

<bound method DataFrame.info of             tavg  tmin  tmax  prcp  snow   wdir  wspd  wpgt    pres  tsun
time                                                                     
2010-01-01  16.3  15.9  17.1   1.3   NaN   45.0   9.7   NaN     NaN   NaN
2010-01-02  17.1  15.4  17.9   0.8   NaN    NaN   7.8   NaN     NaN   NaN
2010-01-03  19.6  16.9  22.6   0.0   NaN    NaN   7.0   NaN     NaN   NaN
2010-01-04  20.1  17.9  22.2   0.0   NaN  108.0   9.2   NaN     NaN   NaN
2010-01-05  22.8  19.8  26.9   0.0   NaN  125.0   9.0   NaN     NaN   NaN
...          ...   ...   ...   ...   ...    ...   ...   ...     ...   ...
2024-11-26  20.9  18.9  23.9   2.3   NaN    2.0  13.3   NaN  1019.0   NaN
2024-11-27  20.4  18.9  21.9   0.0   NaN    3.0  11.5   NaN  1021.3   NaN
2024-11-28  19.7  17.9  21.9   0.0   NaN  354.0  14.2   NaN  1024.2   NaN
2024-11-29  20.8  15.0  25.9   0.0   NaN  335.0   6.8   NaN  1021.2   NaN
2024-11-30  19.0  13.5  24.2   0.0   NaN  154.0   2.5   NaN  1017.1   NaN

[5448

In [249]:
df.apply(pd.isnull).sum()

tavg       0
tmin       7
tmax       6
prcp     703
snow    5448
wdir    1409
wspd      29
wpgt    5448
pres    2133
tsun    5448
dtype: int64

In [250]:
#Tỷ lệ % các giá trị null trong từng cột
missing_percentage = df.apply(pd.isnull).sum()/df.shape[0] * 100
missing_percentage

tavg      0.000000
tmin      0.128488
tmax      0.110132
prcp     12.903818
snow    100.000000
wdir     25.862702
wspd      0.532305
wpgt    100.000000
pres     39.151982
tsun    100.000000
dtype: float64

In [251]:
# Nhận thấy 3 cột snow, wpgt,tsun toàn giá trị null nên ta sẽ bỏ các cột đó 

columns_to_drop = ['snow', 'wpgt','tsun']
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])


In [252]:
df

,tavg,tmin,tmax,prcp,wdir,wspd,pres
time,,,,,,,
2010-01-01,16.3,15.9,17.1,1.3,45.0,9.7,NaN
2010-01-02,17.1,15.4,17.9,0.8,NaN,7.8,NaN
2010-01-03,19.6,16.9,22.6,0.0,NaN,7.0,NaN
2010-01-04,20.1,17.9,22.2,0.0,108.0,9.2,NaN
2010-01-05,22.8,19.8,26.9,0.0,125.0,9.0,NaN
...,...,...,...,...,...,...,...
2024-11-26,20.9,18.9,23.9,2.3,2.0,13.3,1019.0
2024-11-27,20.4,18.9,21.9,0.0,3.0,11.5,1021.3
2024-11-28,19.7,17.9,21.9,0.0,354.0,14.2,1024.2


In [253]:
# Loại bỏ các giá trị ngoại lai dựa trên phương pháp IQR (Interquartile Range)
numeric_columns = df.select_dtypes(include=['float64']).columns
# Hàm loại bỏ giá trị ngoại lai
def remove_outliers(df, columns):
    for col in columns:
        Q1 = df[col].quantile(0.25)  # Quartile 1
        Q3 = df[col].quantile(0.75)  # Quartile 3
        IQR = Q3 - Q1                # Interquartile Range
        lower_bound = Q1 - 1.5 * IQR # Giới hạn dưới
        upper_bound = Q3 + 1.5 * IQR # Giới hạn trên
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df
# Áp dụng hàm để loại bỏ giá trị ngoại lai cho các cột số liệu
df = remove_outliers(df, numeric_columns)

In [254]:
# Kiểm tra số dòng trước và sau khi loại bỏ ngoại lai
original_count = len(df)
new_count = len(df)

original_count, new_count

(1812, 1812)

In [255]:
df = df.interpolate(method='linear', limit_direction='forward', axis=0)

In [256]:
df

,tavg,tmin,tmax,prcp,wdir,wspd,pres
time,,,,,,,
2015-09-05,30.3,26.9,33.9,0.0,128.0,12.5,1008.2
2015-09-09,26.7,25.9,29.9,1.8,68.0,9.4,1010.4
2015-09-13,24.7,23.9,26.9,0.5,31.0,12.8,1013.9
2015-09-14,27.5,22.9,30.9,0.0,8.0,9.4,1011.1
2015-09-16,27.6,24.9,30.9,1.0,11.0,11.3,1009.2
...,...,...,...,...,...,...,...
2024-11-23,23.0,19.9,25.9,0.0,47.0,9.1,1018.7
2024-11-24,22.4,18.9,24.9,0.0,50.0,5.7,1017.8
2024-11-25,24.2,21.9,27.9,0.4,56.0,5.6,1017.0


In [257]:
# Function to classify weather with multiple tags
def classify_weather(row):
    weather_classes = []
    
    if row['prcp'] > 0:
        weather_classes.append('Rainy')
    if row['wspd'] > 10:
        weather_classes.append('Windy')
    if row['tavg'] > 30:
        weather_classes.append('Hot')
    elif 20 <= row['tavg'] <= 30:
        weather_classes.append('Warm')
    elif 10 <= row['tavg'] < 20:
        weather_classes.append('Cool')
    else:
        weather_classes.append('Cold')
    
    return ', '.join(weather_classes)  # Return all classifications as a comma-separated string

# Apply the classification function to each row and create a new 'weather_class' column
df['weather_class'] = df.apply(classify_weather, axis=1)

In [258]:
df

,tavg,tmin,tmax,prcp,wdir,wspd,pres,weather_class
time,,,,,,,,
2015-09-05,30.3,26.9,33.9,0.0,128.0,12.5,1008.2,"Windy, Hot"
2015-09-09,26.7,25.9,29.9,1.8,68.0,9.4,1010.4,"Rainy, Warm"
2015-09-13,24.7,23.9,26.9,0.5,31.0,12.8,1013.9,"Rainy, Windy, Warm"
2015-09-14,27.5,22.9,30.9,0.0,8.0,9.4,1011.1,Warm
2015-09-16,27.6,24.9,30.9,1.0,11.0,11.3,1009.2,"Rainy, Windy, Warm"
...,...,...,...,...,...,...,...,...
2024-11-23,23.0,19.9,25.9,0.0,47.0,9.1,1018.7,Warm
2024-11-24,22.4,18.9,24.9,0.0,50.0,5.7,1017.8,Warm
2024-11-25,24.2,21.9,27.9,0.4,56.0,5.6,1017.0,"Rainy, Warm"
